### Function Calling:
- Function Calling means force the LLM Responce to be in a specific Format, so we can use this response to call a Function(API,Tool) to get a responce then show this responce or return it back to the LLM to generate the Final responce.

### Example : The Stock Price API with Qween2:1.5b LLM

- **Pydentic** is a famous library for data validation,for exemple if I create a instance from the StockInfo class without providing the company attribute i will get a Validation error.

In [3]:
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import List
import yfinance as yf
import instructor


class StockInfo(BaseModel):
    company: str = Field(..., description="Name of the company")
    ticker: str = Field(..., description="Ticker symbol of the company")
    
client = instructor.patch(
    OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="ollama",
    ),
    mode=instructor.Mode.JSON,
)
    

**mode=instructor.Mode.JSON:** This sets the mode to JSON, which likely configures the API to handle JSON input and output. This means that the requests sent to the API and the responses received will be in JSON format.

In [15]:
company = "Apple"

In [16]:
resp = client.chat.completions.create(
    model="qwen2:1.5b",
    messages=[
        {
            "role": "user",
            "content": f"Return the company name and the ticker symbol of the {company}."
        }
    ],
    response_model=StockInfo, # Like We applicate A data Validation test to the response of LLM
    max_retries=10 #  Sets the maximum number of retries for the request in case of transient errors.
)

In [17]:
#Print the reponse of the LLM model
print(resp.model_dump_json(indent=2))

{
  "company": "Apple",
  "ticker": "AAPL"
}


In [18]:
stock = yf.Ticker(resp.ticker)
hist = stock.history(period="1d")
stock_price = hist['Close'].iloc[-1]
print(f"The stock price of the {resp.company} is {stock_price}. USD")

The stock price of the Apple is 230.5399932861328. USD
